<a href="https://colab.research.google.com/github/SarahFeanor/data_modeling_project_ADAtech/blob/main/Prepara_Inserts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<sub> 📁 **Projeto Grupo 1** : Módulo III - Banco de Dados | Turma: 1102 | **Integrantes**: Caio Vieira, Igor Cruz, Jared Leite, Leticia Carneiro, Sarah Rezende </sub>

<sub> **Repositório do Projeto** - [GitHub](https://github.com/SarahFeanor/data_modeling_project_ADAtech/tree/main) </sub>

## 📊 **Scripts para Preparação das Tabelas na Fase de Inserção de Dados**

- Vamos começar visualizando a estrutura atual da tabela.

- O **objetivo principal** é efetuar a transformação dos dados para a etapa de inserção dos dados no Banco.

---

## Scripts

- Leitura e manipulação de dados:

In [ ]:
import pandas as pd

AmostraFipe = pd.read_csv('fipe_2023_amostra.csv', encoding='utf-8', sep=',')
FATO_FIPE = AmostraFipe[['Mes_Ref','Modelo','Ano_Modelo','Preco_Medio','Autenticacao','Data_Consulta']]

- A função `trataTexto` retorna `'NULL'` se a string de entrada for vazia e, caso contrário, retorna a string original entre aspas simples. O exemplo `print(trataTexto('teste'))` imprime `"'teste'"`.

In [ ]:
def trataTexto(txt):
    try:
        if txt == '':
            return 'NULL'
        else:
            return"'"+txt+"'"
    except:
        return 'NULL'
print(trataTexto('teste'))

'teste'


- A função `trataDateRefFipe `converte uma data no formato "mês/ano" (por exemplo, 'janeiro/2000') para o formato "YYYY-MM-DD" e a retorna como uma string.

In [ ]:
def trataDateRefFipe(txt):
    mes_replace= {'janeiro':'01', 'fevereiro':'02', 'março':'03', 'abril':'04', 'maio':'05', 'junho':'06', 'julho':'07',
             'agosto':'08', 'setembro':'09', 'outubro':'10', 'novembro':'11', 'dezembro':'12'
             }
    try:
        mesDesc, ano = txt.split('/')
        mes = mes_replace.get(mesDesc)

        if mes:
            return trataTexto(ano+'-'+mes+'-'+'01')
        else:
            return 'NULL'
    except:
        return 'NULL'

trataDateRefFipe('janeiro/2000')

"'2000-01-01'"

- A função `trataAno_Modelo` processa uma string no formato "Ano Combustível" (por exemplo, '2018 Gasolina') e retorna uma tupla contendo o ano (como um número inteiro) e o tipo de combustível (como uma string).

In [ ]:
def trataAno_Modelo(txt):
    try:
        #ano, combustivel = txt.split(' ')
        lista =  txt.split(' ')
        if lista[0] == 'Zero':
            ano = '0'
            combustivel = lista[2]
        else:
            ano = lista[0]
            combustivel = lista[1]
        return int(ano), trataTexto(combustivel)
    except:
        return 'NULL'

trataAno_Modelo('2018 Gasolina')

(2018, "'Gasolina'")

- A função `trataPreco `processa uma string no formato de preço brasileiro (por exemplo, 'R$ 12.853,00') e retorna o valor como um número inteiro.

In [ ]:
def trataPreco(txt):
    try:
        txt=txt.replace('R$ ','')
        txt=txt.replace(',00','')
        txt=txt.replace('.','')
        return int(txt)
    except:
        return 'NULL'

trataPreco('R$ 12.853,00')

12853


- A função `trataDataConsulta` processa uma string no formato de data e hora em português (por exemplo, 'sábado, 2 de dezembro de 2023 15:30') e a retorna no formato "YYYY-MM-DD HH:MM".

In [ ]:
def trataDataConsulta(txt):
    mes_replace= {'janeiro':'01', 'fevereiro':'02', 'março':'03', 'abril':'04', 'maio':'05', 'junho':'06', 'julho':'07',
             'agosto':'08', 'setembro':'09', 'outubro':'10', 'novembro':'11', 'dezembro':'12'
             }
    try:
        list = txt.split(' ')
        mes = mes_replace.get(list[3])
        dia = int(list[1])
        if dia < 10:
            dia_txt = '0'+list[1]
        else:
            dia_txt = list[1]
        if mes:
            return trataTexto(list[5]+'-'+mes+'-'+dia_txt+' '+list[6]+':00')
        else:
            return 'NULL'
    except:
        return 'NULL'


trataDataConsulta('sábado, 2 de dezembro de 2023 15:30')

"'2023-12-02 15:30:00'"

- Este script Python parece estar gerando comandos SQL do tipo INSERT para a tabela fato_fipe e escrevendo esses comandos em um arquivo chamado insert_fato_fipe.txt.

In [ ]:
inicioInsert='''INSERT INTO fato_fipe
VALUES
'''
with open('insert_fato_fipe.txt', 'a',encoding="utf-8") as file:
    file.write(inicioInsert)
    file.close()


tamanho = len(FATO_FIPE)
for row in FATO_FIPE.iterrows():
    data = trataDateRefFipe(row[1][0])
    modelo = trataTexto(row[1][1])
    ano_modelo, combustivel = trataAno_Modelo(row[1][2])
    preco=trataPreco(row[1][3])
    validacao = trataTexto(row[1][4])
    data_consulta = trataDataConsulta(row[1][5])

    linhaTratada = f"({data},{modelo},{ano_modelo},{combustivel},{preco},{validacao},{data_consulta})"
    if row[0] < tamanho-1:
        linhaTratada = linhaTratada+',\n'
    else:
        linhaTratada = linhaTratada+';'
    with open('insert_fato_fipe.txt', 'a',encoding="utf-8") as file:
        file.write(linhaTratada)
        file.close()


In [ ]:
DIM_FIPE_MARCA_MODELO = AmostraFipe.groupby(['Modelo','Marca','Cod_FIPE']).count()#[['Modelo','Marca','Cod_FIPE']]

DIM_FIPE_MARCA_MODELO

,,,Mes_Ref,Ano_Modelo,Autenticacao,Data_Consulta,Preco_Medio
Modelo,Marca,Cod_FIPE,,,,,
100 2.8 V6,Audi,008030-6,9,9,9,9,9
100 2.8 V6 Avant,Audi,008076-4,6,6,6,6,6
100 S-4 2.2 Avant Turbo,Audi,008031-4,6,6,6,6,6
116iA 1.6 TB 16V 136cv 5p,BMW,009171-5,12,12,12,12,12
118i M Sport 1.5 TB 12V Aut. 5p,BMW,009298-3,9,9,9,9,9
...,...,...,...,...,...,...,...
up! move I MOTION 1.0 T. Flex 12V 3p,VW - VolksWagen,005372-4,6,6,6,6,6
up! move I MOTION 1.0 T. Flex 12V 5p,VW - VolksWagen,005399-6,15,15,15,15,15
up! take 1.0 T. Flex 12V 3p,VW - VolksWagen,005376-7,12,12,12,12,12


- Esse trecho de código em Python está criando comandos SQL do tipo INSERT para a tabela dim_fipe_marca_modelo a partir do DataFrame DIM_FIPE_MARCA_MODELO e escrevendo esses comandos em um arquivo chamado insert_dim_fipe_marca_modelo.txt.

In [ ]:
inicioInsert='''INSERT INTO dim_fipe_marca_modelo
VALUES
'''
with open('insert_dim_fipe_marca_modelo.txt', 'a',encoding="utf-8") as file:
    file.write(inicioInsert)
    file.close()


tamanho = len(DIM_FIPE_MARCA_MODELO)

for index,row in enumerate(DIM_FIPE_MARCA_MODELO.index):
    modelo = trataTexto(row[0])
    marca = trataTexto(row[1])
    cod_fipe = trataTexto(row[2])

    linhaTratada = f"({modelo},{marca},{cod_fipe})"
    if index < tamanho-1:
        linhaTratada = linhaTratada+',\n'
    else:
        linhaTratada = linhaTratada+';'
    with open('insert_dim_fipe_marca_modelo.txt', 'a',encoding="utf-8") as file:
        file.write(linhaTratada)
        file.close()


In [ ]:
AmostraDadosTecnicos = pd.read_csv('tabela_tratada .csv', encoding='utf-8', sep=',')
AmostraDadosTecnicos

,Marca,Familia,Ano_Modelo,Modelo,Configuração,Combustível,Lugares,Portas,Porte,Procedência,Propulsão
0,Ford,Fiesta,2016,Fiesta TITANIUM 1.6 16V Flex Mec.,Hatch,Flex,5,4,Compacto,Nacional,Combustão
1,Honda,Civic,2014,Civic Sedan LXR 2.0 Flexone 16V Aut. 4p,Sedã,Flex,5,4,Médio,Nacional,Combustão
2,Audi,A3 Sedan,2020,A3 Sedan Performance 2.0 TFSI S-tronic,Sedã,Gasolina,5,4,Médio,Nacional,Combustão
3,Ford,Ranger,2003,Ranger XLT 2.8 8v 135cv 4x4 CD TB Diesel,Picape,Diesel,5,4,Médio,Importado,Combustão
4,Audi,Q7,2011,Q7 3.0 V6 TFSI Quat.Tip.5p/ Perf.(Híb.),SUV,Gasolina,7,4,Grande,Importado,NaN
...,...,...,...,...,...,...,...,...,...,...,...
81,Ford,Courier,2001,Courier Sport 1.6 8v,Picape,Gasolina,2,2,Compacto,Nacional,Combustão
82,Audi,Q7,2022,Q7 Perf. Black 3.0 TFSI Quat.Tip.(Híb.),SUV,Gasolina,7,4,Grande,Importado,Híbrido leve
83,BMW,760i,2003,760iL 6.0 V12 445cv/544cv,Sedã,Gasolina,5,4,Grande,Importado,Combustão
84,Audi,A3 Sedan,2016,A3 Sed. Ambition 2.0 TSFI 220cv S-tronic,Sedã,Gasolina,5,4,Médio,Importado,Combustão


In [ ]:
DIM_DADOS_TECNICOS_MODELO = AmostraDadosTecnicos.groupby(['Modelo','Marca','Familia', 'Configuração','Lugares','Portas','Porte','Propulsão'],dropna=False).count()

DIM_DADOS_TECNICOS_MODELO.head(30)

,,,,,,,,Ano_Modelo,Combustível,Procedência
Modelo,Marca,Familia,Configuração,Lugares,Portas,Porte,Propulsão,,,
118i Sport 1.5 TB 12V Aut. 5p,BMW,118i,Hatch,5,4,Médio,Combustão,1,1,1
316i 1.6 TB 16V 136cv 4p,BMW,316i,Sedã,5,4,Médio,Combustão,1,1,1
318i Cabrio 1.8 16V,BMW,318i Cabriolet,Conversível,4,2,Médio,Combustão,1,1,1
320iA,BMW,320i,Sedã,5,4,Médio,Combustão,1,1,1
320iA Modern/Sport TB 2.0/A.Flex/GP 4p,BMW,320i,Sedã,5,4,Médio,Combustão,2,2,2
325i,BMW,325i,Sedã,5,4,Médio,Combustão,1,1,1
325iA,BMW,325i,Sedã,5,4,Médio,Combustão,1,1,1
330i Sport 2.0 TB 16V 4p,BMW,330i,Sedã,5,4,Médio,Combustão,1,1,1
500 Cult 1.4 Flex 8V EVO Dualogic,Fiat,500,Hatch,4,2,Subcompacto,Combustão,1,1,1


- Esse trecho de código em Python está criando comandos SQL do tipo INSERT para a tabela dim_dados_tecnicos_modelo a partir do DataFrame DIM_DADOS_TECNICOS_MODELO e escrevendo esses comandos em um arquivo chamado insert_dim_dados_tecnicos_modelo.txt.

In [ ]:
inicioInsert='''INSERT INTO dim_dados_tecnicos_modelo
VALUES
'''
with open('insert_dim_dados_tecnicos_modelo.txt', 'a',encoding="utf-8") as file:
    file.write(inicioInsert)
    file.close()


tamanho = len(DIM_DADOS_TECNICOS_MODELO)

for index,row in enumerate(DIM_DADOS_TECNICOS_MODELO.index):
    modelo = trataTexto(row[0])
    marca = trataTexto(row[1])
    familia = trataTexto(row[2])
    configuracao = trataTexto(row[3])
    lugares = row[4]
    portas = row[5]
    porte = trataTexto(row[6])
    propulsao = trataTexto(row[7])


    linhaTratada = f"({modelo},{marca},{familia},{configuracao},{lugares},{portas},{porte},{propulsao})"
    if index < tamanho-1:
        linhaTratada = linhaTratada+',\n'
    else:
        linhaTratada = linhaTratada+';'
    with open('insert_dim_dados_tecnicos_modelo.txt', 'a',encoding="utf-8") as file:
        file.write(linhaTratada)
        file.close()


In [ ]:
DIM_DADOS_TECNICOS_ANO_MODELO = AmostraDadosTecnicos.groupby(['Modelo','Ano_Modelo','Combustível', 'Procedência'],dropna=False).count()

DIM_DADOS_TECNICOS_ANO_MODELO.head(30)

Marca  \
Modelo                                   Ano_Modelo Combustível Procedência          
118i Sport 1.5 TB 12V Aut. 5p            2020       Gasolina    Importado        1   
316i 1.6 TB 16V 136cv 4p                 2015       Gasolina    Importado        1   
318i Cabrio 1.8 16V                      1994       Gasolina    Importado        1   
320iA                                    2003       Gasolina    Importado        1   
320iA Modern/Sport TB 2.0/A.Flex/GP 4p   2018       Flex        Nacional         1   
                                         2020       Gasolina    Nacional         1   
325i                                     1995       Gasolina    Importado        1   
325iA                                    2012       Gasolina    Importado        1   
330i Sport 2.0 TB 16V 4p                 2020       Gasolina    Nacional         1   
500 Cult 1.4 Flex 8V EVO Dualogic        2013       Flex        Importado        1   
535iA M Sport 3.0 24V 306cv Bi-Turbo     2016       Gasolina    Importado        1   
550iA 4.8 32V                            2009       Gasolina    Importado        1   
760iL 6.0 V12 445cv/544cv                2003       Gasolina    Importado        1   
80 2.0 Avant                             1995       Gasolina    Importado        1   
A3 Sed. Ambition 2.0 TSFI 220cv S-tronic 2016       Gasolina    Importado        1   
A3 Sed.Perf. Black 2.0 TFSI S-tr. (Hib.) 2021       Gasolina    Nacional         1   
A3 Sedan Performance 2.0 TFSI S-tronic   2020       Gasolina    Nacional         1   
A4 2.8 Avant V6 30V Tiptronic            2000       Gasolina    Importado        1   
ARGO DRIVE 1.0 6V Flex                   2021       Flex        Nacional         1   
ARGO DRIVE 1.3 8V Flex                   2020       Flex        Nacional         1   
CITY Hatchback Touring 1.5 Flex 16V Aut  2022       Flex        Nacional         1   
CITY Sedan DX 1.5 Flex 16V Mec.          2017       Flex        Nacional         1   
CR-V 2.0 16V Aut.                        2007       Gasolina    Importado        1   
CR-V EXL 2.0 16V 4WD/2.0 Flexone Aut.    2008       Gasolina    Importado        1   
                                         2016       Flex        Importado        1   
Civic Sedan LXR 2.0 Flexone 16V Aut. 4p  2014       Flex        Nacional         1   
Coupe 16V                                1996       Gasolina    Importado        1   
Courier Sport 1.6 8v                     2001       Gasolina    Nacional         1   
Doblo ATTRACTIVE 1.4 Fire Flex 8V 5p     2014       Flex        Nacional         1   
Doblo Adv/Adv TRYON/LOCKER 1.8 Flex\t    2019       Flex        Nacional         1   

                                                                             Familia  \
Modelo                                   Ano_Modelo Combustível Procedência            
118i Sport 1.5 TB 12V Aut. 5p            2020       Gasolina    Importado          1   
316i 1.6 TB 16V 136cv 4p                 2015       Gasolina    Importado          1   
318i Cabrio 1.8 16V                      1994       Gasolina    Importado          1   
320iA                                    2003       Gasolina    Importado          1   
320iA Modern/Sport TB 2.0/A.Flex/GP 4p   2018       Flex        Nacional           1   
                                         2020       Gasolina    Nacional           1   
325i                                     1995       Gasolina    Importado          1   
325iA                                    2012       Gasolina    Importado          1   
330i Sport 2.0 TB 16V 4p                 2020       Gasolina    Nacional           1   
500 Cult 1.4 Flex 8V EVO Dualogic        2013       Flex        Importado          1   
535iA M Sport 3.0 24V 306cv Bi-Turbo     2016       Gasolina    Importado          1   
550iA 4.8 32V                            2009       Gasolina    Importado          1   
760iL 6.0 V12 445cv/544cv                2003       Gasolina    Importado          1   
80 2

- Esse trecho de código em Python está criando comandos SQL do tipo INSERT para a tabela dim_dados_tecnicos_ano_modelo a partir do DataFrame DIM_DADOS_TECNICOS_ANO_MODELO e escrevendo esses comandos em um arquivo chamado insert_dim_dados_tecnicos_ano_modelo.txt.

In [ ]:
inicioInsert='''INSERT INTO dim_dados_tecnicos_ano_modelo
VALUES
'''
with open('insert_dim_dados_tecnicos_ano_modelo.txt', 'a',encoding="utf-8") as file:
    file.write(inicioInsert)
    file.close()


tamanho = len(DIM_DADOS_TECNICOS_ANO_MODELO)

for index,row in enumerate(DIM_DADOS_TECNICOS_ANO_MODELO.index):
    modelo = trataTexto(row[0])
    ano_modelo = row[1]
    combustivel = trataTexto(row[2])
    if row[2]=='Diesel' or row[2]=='Álcool':
        combustivel_fipe = combustivel
    else:
        combustivel_fipe = "'Gasolina'"
    procedencia = trataTexto(row[3])

    linhaTratada = f"({modelo},{ano_modelo},{combustivel},{combustivel_fipe},{procedencia})"
    if index < tamanho-1:
        linhaTratada = linhaTratada+',\n'
    else:
        linhaTratada = linhaTratada+';'
    with open('insert_dim_dados_tecnicos_ano_modelo.txt', 'a',encoding="utf-8") as file:
        file.write(linhaTratada)
        file.close()


In [ ]:
AmostraSalarioMinimo = pd.read_csv('Ipeadata_Tratado.csv', encoding='utf-8', sep=',')
AmostraSalarioMinimo

,Data,Salario_Minimo_Vigente
0,2000-01-01,136
1,2000-02-01,136
2,2000-03-01,136
3,2000-04-01,151
4,2000-05-01,151
...,...,...
283,2023-08-01,1320
284,2023-09-01,1320
285,2023-10-01,1320
286,2023-11-01,1320


- Este trecho de código em Python está criando comandos SQL do tipo INSERT para a tabela fato_salario_minimo a partir do DataFrame AmostraSalarioMinimo e escrevendo esses comandos em um arquivo chamado insert_fato_salario_minimo.txt.

In [ ]:
inicioInsert='''INSERT INTO fato_salario_minimo
VALUES
'''
with open('insert_fato_salario_minimo.txt', 'a',encoding="utf-8") as file:
    file.write(inicioInsert)
    file.close()


tamanho = len(AmostraSalarioMinimo)
for row in AmostraSalarioMinimo.iterrows():
    data = trataTexto(row[1][0])
    valor = row[1][1]

    linhaTratada = f"({data},{valor})"
    if row[0] < tamanho-1:
        linhaTratada = linhaTratada+',\n'
    else:
        linhaTratada = linhaTratada+';'
    with open('insert_fato_salario_minimo.txt', 'a',encoding="utf-8") as file:
        file.write(linhaTratada)
        file.close()
